# Подключение библиотек

In [2]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
import time
import struct
import os

# Подключимся к MetaTrader 5

In [3]:
if not mt5.initialize():
    print("initialize() failed")
    quit()

In [4]:
def take_rates(): 
    eurusd_rates = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_M1, 0, 15)
    gbpusd_rates = mt5.copy_rates_from_pos("AUDUSD", mt5.TIMEFRAME_M1, 0, 15)
    return eurusd_rates, gbpusd_rates

In [18]:
def calculation(rate1, rate2, lotaud):
    spread = rate1 - rate2 * lotaud * 10 # Вычисляем спред
    spread = np.array(spread)
    median = np.median(spread)
    Open_up = list()
    Open_down = list()
    Open_UP =0
    Open_DOWN = 0
    #Вычисляем верхнюю и нижнюю границу
    for i in range(0,len(spread)):
        if spread[i] > median:
            Open_up.append(spread[i])
        if spread[i] < median:
            Open_down.append(spread[i])
    Open_UP =np.median(Open_up)
    Open_DOWN = np.median(Open_down)
    return Open_UP, Open_DOWN, median

# Подготавливаем все запросы
## Нам нужно 8 запросов: 4 для открытия позиции, 4 для закрытия позиции.

In [6]:
symbol_Euro = "EURUSD"
symbol_aud = "AUDUSD"
symbol_Euro_info =mt5.symbol_info(symbol_Euro)
symbol_Aud_info =mt5.symbol_info(symbol_aud)
loteuro = 0.1
lotaud = round(loteuro * symbol_Euro_info.session_close / symbol_Aud_info.session_close, 2)
point_euro = mt5.symbol_info(symbol_Euro).point
point_Aud = mt5.symbol_info(symbol_aud).point
deviation = 1

# #Краткое описание каждого аргумента

# "action": mt5.TRADE_ACTION_DEAL,      # Совершить сделку прямо сейчас

# "symbol": symbol_Euro,                # Валюта сделки

# "volume": lot,                        # Объем который мы покупаем: У нас равен 1

# "type": mt5.ORDER_TYPE_SELL,          # Покупаем или продаём

# "deviation": deviation,               # Максимальное отклонение от заданной цены

# "magic": 123456,                      # Идентификатор эксперта. 
#                                       #Позволяет организовать аналитическую обработку торговых ордеров.    

# "comment": "Какой либо комментарий",  # Комменатрий к запросу

# "type_time": mt5.ORDER_TIME_GTC,      # Тип ордера по истечению: У нас Ордер будет находиться 
#                                       # в очереди до тех пор, пока не будет снят

# "type_filling": mt5.ORDER_FILLING_FOK,# Не будет продажи, если нет необходимого обьема 
    
    
request_euro_sell_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_Euro,
"volume": loteuro, 
"type": mt5.ORDER_TYPE_SELL,
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, купили евро",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,
    

}
request_aud_buy_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_aud,
"volume": lotaud,
"type": mt5.ORDER_TYPE_BUY,
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, продали фунт",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,      
}

request_euro_buy_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_Euro,
"volume": loteuro,
"type": mt5.ORDER_TYPE_BUY, 
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, купили евро",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,  
}

request_aud_sell_open = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_aud,
"volume": lotaud,
"type": mt5.ORDER_TYPE_SELL, 
"deviation": deviation,
"magic": 123456,
"comment": "Открыли позицию, продали фунт",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,      
}

# В запросах для закрытия позиции есть ещё один аргумент

#"position": order_euro_sell.order,                     # Указывает какую позицию мы закрывем


# Чтобы запросы закрытия позции знали, что закрыват откроем предыдущие позции
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
order_euro_buy_open = mt5.order_send(request_euro_buy_open)
order_euro_sell_open = mt5.order_send(request_euro_sell_open)
order_aud_buy_open = mt5.order_send(request_aud_buy_open)
order_aud_sell_open = mt5.order_send(request_aud_sell_open)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

request_euro_buy_close = { 
"action": mt5.TRADE_ACTION_DEAL, 
"symbol": symbol_Euro, 
"volume": loteuro, 
"type": mt5.ORDER_TYPE_BUY,
"position": order_euro_sell_open.order,
"deviation": deviation, 
"magic": 123456, 
"comment": "купили евро", 
"type_time": mt5.ORDER_TIME_GTC, 
"type_filling": mt5.ORDER_FILLING_FOK, 
}
request_aud_sell_close = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_aud,
"volume": lotaud,
"type": mt5.ORDER_TYPE_SELL,
"position": order_aud_buy_open.order,
"deviation": deviation,
"magic": 123456,
"comment": "продажа фунта",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,   
}
request_euro_sell_close = { 
"action": mt5.TRADE_ACTION_DEAL, 
"symbol": symbol_Euro, 
"volume": loteuro, 
"type": mt5.ORDER_TYPE_SELL,
"position": order_euro_buy_open.order,
"deviation": deviation, 
"magic": 123456, 
"comment": "купили евро", 
"type_time": mt5.ORDER_TIME_GTC, 
"type_filling": mt5.ORDER_FILLING_FOK, 
}
request_aud_buy_close = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_aud,
"volume": lotaud,
"type": mt5.ORDER_TYPE_BUY,
"position": order_aud_sell_open.order,
"deviation": deviation,
"magic": 123456,
"comment": "продажа фунта",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,   
}

# Функции для открытия и закрытия позиции

In [7]:
def request_open_up_pos(): 
    result = mt5.order_send(request_euro_sell_open) 
    result2 = mt5.order_send(request_aud_buy_open)
    print('Открыл сверху')

In [8]:
def request_open_down_pos(): 
    result =mt5.order_send(request_euro_buy_open) 
    result2 = mt5.order_send(request_aud_sell_open)
    print('Открыл снизу')

In [9]:
def request_close_up_pos():
    result = mt5.order_send(request_euro_buy_close) 
    result2 = mt5.order_send(request_aud_sell_close)
    print('Закрыл сверху')

In [10]:
def request_close_down_pos():
    result = mt5.order_send(request_euro_sell_close) 
    result2 = mt5.order_send(request_aud_buy_close)
    print('Закрыл снизу')

# Бесконечный цикл 1:
## Отрывает позицию в большом объеме и закрывает её как достигнет условие(Условие: тики раз в 10 мин за последний год)

In [25]:
#Достаём значения для открытия и закрытия позиции
values = calculation(take_rates()[0]['close'], take_rates()[1]['close'], lotaud)
Open_Up = values[0]
Open_Down = values[1]
Close = values[2]
print(Open_Up, Open_Down, Close)
#Некоторые переменные для подсчитывания открытых позиции и прибыли
Pos_Up = False
Pos_Down = False
quantity_open = 0
quantity_close = 0
balance_old = mt5.account_info().balance
while(True):
    if datetime.now().minute % 15 == 0 and datetime.now().second == 0:
        if Pos_Up == True:
            Pos_Up = False
            request_close_up_pos()
        if Pos_Down == True:
            Pos_Down = False
            request_close_down_pos()
        values = calculation(take_rates()[0]['close'], take_rates()[1]['close'], lotaud)
        Open_Up = values[0]
        Open_Down = values[1]
        Close = values[2]
        print(Open_Up, Open_Down, Close)
        time.sleep(1)
    last_tick_AUDUSD_ask = mt5.symbol_info_tick("AUDUSD").ask
    last_tick_EURUSD_ask = mt5.symbol_info_tick("EURUSD").ask
    last_tick_AUDUSD_bid = mt5.symbol_info_tick("AUDUSD").bid
    last_tick_EURUSD_bid = mt5.symbol_info_tick("EURUSD").bid
    
    if Pos_Up == False: #Купить дешевую продать дорогую
        last_spread_up = last_tick_EURUSD_bid- last_tick_AUDUSD_ask * lotaud * 10
        
    if Pos_Down == False:#Купить дешевую продать дорогую
        last_spread_down = last_tick_EURUSD_ask - last_tick_AUDUSD_bid * lotaud * 10
        
    if Pos_Up == True: #Купить дешевую продать дорогую
        last_spread_up = last_tick_EURUSD_ask - last_tick_AUDUSD_bid * lotaud * 10
        
    if Pos_Down == True:#Купить дешевую продать дорогую
        last_spread_down = last_tick_EURUSD_bid - last_tick_AUDUSD_ask * lotaud * 10

    #print(last_spread_up)

    if last_spread_up > Open_Up and Pos_Up == False:
        Pos_Up = True
        request_open_up_pos()    

    if last_spread_down < Open_Down and Pos_Down == False:
        Pos_Down = True
        request_open_down_pos()
        
    if last_spread_up <= Close and Pos_Up == True:
        Pos_Up = False
        request_close_up_pos()
        
    if last_spread_down >= Close and Pos_Down == True:
        Pos_Down = False
        request_close_down_pos()

0.02729500000000029 0.02694500000000022 0.027140000000000164
Открыл сверху
Закрыл сверху
0.027695000000000025 0.02719999999999989 0.027349999999999985
0.027730000000000254 0.02751999999999999 0.027665000000000273
Открыл снизу
Закрыл снизу
0.027410000000000156 0.027344999999999953 0.027370000000000116
Открыл снизу
Закрыл снизу
Открыл сверху


KeyboardInterrupt: 

In [108]:
mt5.shutdown()

True

#  Далее черновой код не использующийся в основной программе